In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define the neural network model
class CardDetector(nn.Module):
    def __init__(self):
        super(CardDetector, self).__init__()
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # Define the pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # Define the fully connected layers
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)
    
    # Define the forward pass of the neural network
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the neural network model
model = CardDetector()

# Define the image transformations to be applied to the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224 pixels
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the pixel values to have mean=0.5 and std=0.5
])

# Load the training dataset and create a dataloader
train_dataset = datasets.ImageFolder("train/", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load the validation dataset and create a dataloader
val_dataset = datasets.ImageFolder("val/", transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the loss function to be used during training
criterion = nn.CrossEntropyLoss()

# Define the optimizer to be used during training
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Set the model to training mode
        model.train()

        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the model parameters
        optimizer.step()

        # Print statistics
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Switch to evaluation mode and calculate accuracy on validation set
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f'Validation accuracy: {accuracy:.2f}%')



tensor([[2.2946e-01, 6.6674e-01, 6.0311e-01],
        [8.9196e-01, 3.0727e-01, 6.2293e-04],
        [1.6974e-01, 1.8591e-01, 4.0299e-01],
        [5.8913e-02, 5.8426e-01, 4.2495e-01],
        [3.4141e-01, 2.9352e-01, 2.4786e-01]])
